In [2]:
############### EEG dataset #################

# Disable info messages from the tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom
import pickle

In [3]:
#Load train data
with open('train', 'rb') as fo:         #Open file
    d = pickle.load(fo, encoding='bytes')       #Load pickle data
    train_data = d['data']                      #Load train data
    train_label = d['label']                    #Load train label
 
 
with open('test', 'rb') as fo:          #Open file
    d = pickle.load(fo, encoding='bytes')       #Load pickle data
    test_data = d['data']                       #Load test data
    test_label = d['label']                     #Load test data
print(train_data.shape,test_data.shape) # (84420,310) # (58128,310)
#Merge train and test data in order to scale them
X = np.concatenate([train_data,test_data])
# scaling the inputs from 0 to 1
X = MinMaxScaler().fit_transform(X)
Y = np.concatenate([train_label,test_label])
classes = len(np.unique(Y))
Y = LabelEncoder().fit_transform(Y)
Y = to_categorical(Y,classes)
print(X.shape,Y.shape) #(142548,310)
# split into train and test data
X_train = X[:84420]
X_test =  X[84420:]
Y_train = Y[:84420]
Y_test = Y[84420:]

(84420, 310) (58128, 310)
(142548, 310) (142548, 3)


In [8]:
dim = X_train.shape[1]
x = int(dim**(1/2)) +1
f = x*x
model = Sequential()
model.add(Dense(f-dim,name='feature', activation='relu',input_shape=(dim,)))
model.add(Dense(classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
# here, inputs and labels are same
model.fit(X_train,Y_train,epochs=10,batch_size=256,verbose=0)
extract = Model(model.inputs, model.get_layer('feature').output)
# predict whole inputs through it
x1 = extract.predict(X_train)
x2 = extract.predict(X_test)
# concatenate on horizontal axis
X1 = np.concatenate((X_train, x1), axis=1) 
X2 = np.concatenate((X_test, x2), axis=1) 
print(X1.shape,X2.shape)
train = X1.reshape(X1.shape[0],x,x)
test = X2.reshape(X2.shape[0],x,x)
print(train.shape,test.shape)

(84420, 324) (58128, 324)
(84420, 18, 18) (58128, 18, 18)


In [9]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(train)):
    img = Image.fromarray(train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(test)):
    img = Image.fromarray(test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(84420, 32, 32, 3) (58128, 32, 32, 3)


In [10]:
############### Feature-extraction ###############
# pretrained model for FE
from keras import callbacks 
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
#model.fit(x_train,Y_train,batch_size=1024, epochs=20,verbose=1,validation_data=(x_test,Y_test))
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  mode ="min", patience = 5,  restore_best_weights = True) 
history = model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test), callbacks =[earlystopping])

Epoch 1/20
330/330 [==============================] - 34s 102ms/step - loss: 0.1145 - accuracy: 0.9578 - val_loss: 2.4341 - val_accuracy: 0.5952
Epoch 2/20
330/330 [==============================] - 27s 81ms/step - loss: 0.0286 - accuracy: 0.9907 - val_loss: 2.1453 - val_accuracy: 0.7506
Epoch 3/20
330/330 [==============================] - 27s 81ms/step - loss: 0.0153 - accuracy: 0.9946 - val_loss: 2.2467 - val_accuracy: 0.7469
Epoch 4/20
330/330 [==============================] - 27s 81ms/step - loss: 0.0128 - accuracy: 0.9963 - val_loss: 4.6705 - val_accuracy: 0.6479
Epoch 5/20
330/330 [==============================] - 27s 81ms/step - loss: 0.0116 - accuracy: 0.9964 - val_loss: 3.3065 - val_accuracy: 0.7265
Epoch 6/20
330/330 [==============================] - 27s 82ms/step - loss: 7.6831e-04 - accuracy: 0.9998 - val_loss: 2.4510 - val_accuracy: 0.7722
Epoch 7/20
330/330 [==============================] - 27s 82ms/step - loss: 2.5689e-05 - accuracy: 1.0000 - val_loss: 2.8312 - val_

In [11]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

2639/2639 [==============================] - 26s 10ms/step - loss: 0.0693 - accuracy: 0.9769
training accuracy:  0.9769130349159241
testing accuracy:  0.7506193518638611
